In [10]:
import backtrader as bt
import pandas as pd
import datetime
import logging

# Configuração básica de log para vermos as ordens sendo executadas
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

class AurumData(bt.feeds.PandasData):
    """
    1. Resolve a falta de Open/High/Low usando Adj Close.
    2. Adiciona Volatilidade (disponível no seu dataset).
    """
    lines = ('aurum_score', 'roe', 'roic', 'volatility',)

    params = (
        ('datetime', None),
        
        ('open', 'Adj Close'),  
        ('high', 'Adj Close'),
        ('low', 'Adj Close'),
        ('close', 'Adj Close'), 
        
        ('volume', -1), 
        ('openinterest', None),
        
        ('aurum_score', 'aurum_quality_score'),
        ('roe', 'ROE'),
        ('roic', 'ROIC'),
        ('volatility', 'VOLATILIDADE'),
    )

In [11]:
# --- ESTRATÉGIA DE RANKING ---
class AurumRankingStrategy(bt.Strategy):
    params = (
        ('top_n', 10),           
        ('rebalance_days', 1),  
        ('reserve_cash', 0.05), 
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} | {txt}')

    def __init__(self):
        self.inds = {}
        for d in self.datas:
            self.inds[d] = {
                'score': d.aurum_score,
                'name': d._name
            }
        self.timer_count = 0

    def next(self):
        self.timer_count += 1
        if self.timer_count < self.params.rebalance_days:
            return
        
        self.timer_count = 0
        
        self.log(f'--- INICIANDO REBALANCEAMENTO (Cash: {self.broker.get_cash():.2f}) ---')

        candidates = []
        for d in self.datas:
            if len(d) > 0 and d.close[0] > 0 and d.aurum_score[0] > 0:
                candidates.append((d, d.aurum_score[0]))

        candidates.sort(key=lambda x: x[1], reverse=True)

        top_stocks = [x[0] for x in candidates[:self.params.top_n]]
        top_names = [d._name for d in top_stocks]
        
        self.log(f'TOP {self.params.top_n} ATUAIS: {top_names}')

        
        for d in self.datas:
            if self.getposition(d).size > 0:
                if d not in top_stocks:
                    self.log(f'VENDENDO {d._name} (Saiu do Ranking)')
                    self.close(d) 

        target_pct = (1.0 - self.params.reserve_cash) / self.params.top_n
        
        for d in top_stocks:
            self.order_target_percent(d, target=target_pct)

In [ ]:
def filtrar_tickers_problematicos(df, limite_alta=1.0, limite_baixa=-0.8):
    """
    Remove tickers que tenham variações mensais absurdas (>100% ou <-80%).
    Isso elimina erros de split/agrupamento do Yahoo Finance.
    """
    print("\n🧹 Iniciando Filtro de Sanidade dos Dados...")
    tickers_originais = df['ticker'].unique()
    
    # Calcula variação percentual para cada ticker
    # Usamos transform para manter o shape original e facilitar o filtro
    df['retorno'] = df.groupby('ticker')['Adj Close'].pct_change()
    
    # Identifica tickers com retornos impossíveis
    bad_tickers = df[
        (df['retorno'] > limite_alta) | 
        (df['retorno'] < limite_baixa)
    ]['ticker'].unique()
    
    if len(bad_tickers) > 0:
        print(f"🚫 BANINDO {len(bad_tickers)} tickers com dados corrompidos:")
        print(f"   -> {list(bad_tickers)}")
        
        # Filtra o DataFrame
        df_limpo = df[~df['ticker'].isin(bad_tickers)].copy()
        print(f"✅ Tickers restantes: {len(df_limpo['ticker'].unique())} de {len(tickers_originais)}")
        return df_limpo
    else:
        print("✅ Nenhum erro grave detectado nos dados.")
        return df

In [ ]:
def run_strategy():
    print("--- 🦁 INICIANDO BACKTEST AURUM (COM FILTRO DE DADOS) ---")
    cerebro = bt.Cerebro()

    path_data = "../data/aurum_master_features.parquet"
    try:
        df_master = pd.read_parquet(path_data)
        df_master['date'] = pd.to_datetime(df_master['date'])
        df_master = df_master.sort_values('date')
    except Exception as e:
        print(f"Erro: {e}"); return

    # --- APLICAÇÃO DO FILTRO ---
    df_master = filtrar_tickers_problematicos(df_master)
    # ---------------------------

    start_date = '2019-01-01'
    end_date = '2023-12-31'
    mask = (df_master['date'] >= start_date) & (df_master['date'] <= end_date)
    df_filtered = df_master.loc[mask]
    
    tickers = df_filtered['ticker'].unique()
    print(f"\nCarregando {len(tickers)} feeds para o Backtrader...")

    for ticker in tickers:
        df_ticker = df_filtered[df_filtered['ticker'] == ticker].copy()
        df_ticker = df_ticker.set_index('date').sort_index()

        if len(df_ticker) < 6: continue 

        data_feed = AurumData(
            dataname=df_ticker, name=ticker,
            fromdate=pd.to_datetime(start_date), todate=pd.to_datetime(end_date)
        )
        cerebro.adddata(data_feed)

    cerebro.addstrategy(AurumRankingStrategy, top_n=5)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0005)

    print(f'\n💰 Saldo Inicial: R$ {cerebro.broker.getvalue():,.2f}')
    cerebro.run()
    print(f'💰 Saldo Final:   R$ {cerebro.broker.getvalue():,.2f}')
    
    # cerebro.plot() # Descomente se quiser o gráfico

if __name__ == '__main__':
    run_strategy()

--- 🦁 INICIANDO BACKTEST AURUM (VERSÃO FINAL) ---
Tickers no período: 94
Adicionando feeds...
Feeds carregados: 30
Saldo Inicial: 100000.00
2019-01-01 | --- INICIANDO REBALANCEAMENTO (Cash: 100000.00) ---
2019-01-01 | TOP 5 ATUAIS: ['CSNA3.SA', 'WEGE3.SA', 'SBSP3.SA', 'TEND3.SA', 'MGLU3.SA']
2019-01-31 | --- INICIANDO REBALANCEAMENTO (Cash: 9602.63) ---
2019-01-31 | TOP 5 ATUAIS: ['CSNA3.SA', 'WEGE3.SA', 'SBSP3.SA', 'TEND3.SA', 'MGLU3.SA']
2019-02-01 | --- INICIANDO REBALANCEAMENTO (Cash: 4983.26) ---
2019-02-01 | TOP 5 ATUAIS: ['CSNA3.SA', 'WEGE3.SA', 'SBSP3.SA', 'TEND3.SA', 'MGLU3.SA']
2019-02-28 | --- INICIANDO REBALANCEAMENTO (Cash: 4983.26) ---
2019-02-28 | TOP 5 ATUAIS: ['CSNA3.SA', 'WEGE3.SA', 'SBSP3.SA', 'TEND3.SA', 'MGLU3.SA']
2019-03-01 | --- INICIANDO REBALANCEAMENTO (Cash: 5089.96) ---
2019-03-01 | TOP 5 ATUAIS: ['CSNA3.SA', 'WEGE3.SA', 'SBSP3.SA', 'TEND3.SA', 'MGLU3.SA']
2019-03-31 | --- INICIANDO REBALANCEAMENTO (Cash: 5089.96) ---
2019-03-31 | TOP 5 ATUAIS: ['CSNA3.SA', 

In [13]:
import pandas as pd
import backtrader as bt
import os
import sys

# --- CONFIGURAÇÕES ---
FILE_PATH = "../data/aurum_master_features.parquet"

def debug_data_loading():
    print("🔴 1. INICIANDO DIAGNÓSTICO DE DADOS...")
    
    # 1. Checagem Física do Arquivo
    if not os.path.exists(FILE_PATH):
        print(f"❌ ERRO CRÍTICO: O arquivo não existe no caminho: {os.path.abspath(FILE_PATH)}")
        print("   Verifique se a pasta '../data' está correta em relação a onde você está rodando este script.")
        return None
    print(f"✅ Arquivo encontrado em: {os.path.abspath(FILE_PATH)}")

    # 2. Carregamento com Pandas
    try:
        df = pd.read_parquet(FILE_PATH)
        print(f"✅ Arquivo Parquet carregado. Shape: {df.shape}")
    except Exception as e:
        print(f"❌ ERRO AO LER PARQUET: {e}")
        return None

    # 3. Validação de Colunas
    required_cols = ['date', 'ticker', 'Adj Close', 'aurum_quality_score']
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"❌ COLUNAS FALTANDO: {missing}")
        print(f"   Colunas disponíveis: {df.columns.tolist()}")
        return None
    print("✅ Colunas essenciais presentes.")

    # 4. Validação de Tipos de Dados (O erro mais comum no Backtrader!)
    print("\n🔍 Verificando Tipos de Dados:")
    print(df.dtypes)
    
    # Check Data
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        print("⚠️ AVISO: Coluna 'date' não é datetime. Tentando converter...")
        df['date'] = pd.to_datetime(df['date'])
    
    # Check Float
    if not pd.api.types.is_float_dtype(df['Adj Close']):
        print("❌ ERRO: 'Adj Close' não é float (número decimal). O Backtrader não vai ler!")
        return None

    # 5. Filtragem de Teste
    print("\n🔍 Testando Filtro de Datas:")
    df = df.sort_values('date')
    start_date = '2020-01-01'
    mask = df['date'] >= start_date
    df_filtered = df.loc[mask]
    
    unique_tickers = df_filtered['ticker'].unique()
    print(f"   Tickers encontrados após 2020: {len(unique_tickers)}")
    
    if len(unique_tickers) == 0:
        print("❌ ERRO: O filtro de data retornou 0 tickers. Verifique as datas no seu Parquet.")
        print(f"   Data Mínima no arquivo: {df['date'].min()}")
        print(f"   Data Máxima no arquivo: {df['date'].max()}")
        return None

    # Pega o primeiro ticker que tiver dados suficientes
    test_ticker = unique_tickers[0]
    df_one = df_filtered[df_filtered['ticker'] == test_ticker].copy()
    print(f"✅ Selecionado para teste do Backtrader: {test_ticker} ({len(df_one)} linhas)")

    return df_one, test_ticker

def test_backtrader_engine(df, ticker_name):
    print("\n🔴 2. TESTANDO MOTOR BACKTRADER (SIMPLES)...")
    
    # Prepara o DF para o Backtrader (Index deve ser data)
    df = df.set_index('date').sort_index()
    
    # Preenchimento básico
    for col in ['Open', 'High', 'Low']:
        if col not in df.columns: df[col] = df['Adj Close']
    if 'Volume' not in df.columns: df['Volume'] = 0

    cerebro = bt.Cerebro()

    # Data Feed Genérico (Sem as colunas personalizadas por enquanto, só pra ver se roda)
    data = bt.feeds.PandasData(
        dataname=df,
        name=ticker_name,
        open='Open', high='High', low='Low', close='Adj Close', volume='Volume'
    )
    
    cerebro.adddata(data)
    
    print("   Iniciando Cerebro.run()...")
    try:
        cerebro.run()
        print("✅ SUCESSO! O Backtrader processou os dados sem erro.")
    except Exception as e:
        print(f"❌ O BACKTRADER FALHOU: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    result = debug_data_loading()
    if result is not None:
        df_test, ticker = result
        test_backtrader_engine(df_test, ticker)
    else:
        print("\n❌ ABORTADO: Corrija os erros de dados acima antes de tentar o Backtrader.")

🔴 1. INICIANDO DIAGNÓSTICO DE DADOS...
✅ Arquivo encontrado em: c:\Users\kaike\projeto_aurum\aurum\data\aurum_master_features.parquet
✅ Arquivo Parquet carregado. Shape: (18879, 55)
✅ Colunas essenciais presentes.

🔍 Verificando Tipos de Dados:
date                                         datetime64[ns]
ticker                                               object
Adj Close                                           float64
CNPJ_CIA                                             object
DENOM_CIA                                            object
date_balanco                                 datetime64[ns]
Custo dos Bens e/ou Serviços Vendidos               float64
EBIT                                                float64
EBT                                                 float64
Lucro Bruto                                         float64
Lucro Líquido Consolidado                           float64
Receita Líquida                                     float64
Ativo Circulante                   